In [12]:
using LinearAlgebra 
using BenchmarkTools

In [13]:
##
# Funciones para construir el sistema Ax = b. Se debe usar un número entero par
# mayor que cero.
# #

##
# Función que construye la matriz "A".
##
function matrizSistema( n )
    M = Tridiagonal(-1*ones(n-1), 3*ones(n), -1*ones(n-1))
    M = M + zeros(n,n)
    for i = 1:n
        if i != n/2 && i != n/2 + 1
            M[i,n+1-i] = 0.5
        end
    end
    return M
end

matrizSistema (generic function with 1 method)

In [14]:
##
# Función que construye el vector "b".
## 
function vectorSistema(n)
    b = zeros(n)
    b[1] = 5/2
    b[n] = 5/2
    for i = 2:n-1
        if i == n/2 || i == n/2+1
            b[i] = 1
        else
            b[i] = 3/2
        end
    end
    return b
end

vectorSistema (generic function with 1 method)

In [15]:
n = 30
A = matrizSistema(n)
b = vectorSistema(n)
nA = norm(A);
typeof(A)

Matrix{Float64} (alias for Array{Float64, 2})

## Implementaciones
A continuación se muestran diversas versiones del Método de Monte Carlo Cadenas de MArkov **(MCCM)** con la finalidad de evaluar la eficiencia y escalabilidad.

### MCCM con matriz de probabilidad in situ.
Esta primera versión del método crea las matrices auxiliares y posteriormente son utilizadas como variables globales. Adicionalmente, para elegir los elementos de `T, f` y `P, Pi` necesarios para construir las cadenas de Markov se hace una suma acumulada de probabilidades cada vez que se calcula un elemento de cada cadena.

In [16]:
M = diagm(0 => diag(A))
N = M-A
T = inv(M) * N
f = inv(M) * b
nT, mT = size(T);

In [17]:
S = fill(0, nT)
P = fill(0., nT, mT) 
[S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0]
[P[i,j]= 1/S[i] for i in 1:nT, j in 1:mT if T[i,j] != 0 ]
Pi = [1/nT for i in 1:nT];

In [18]:
ϵ = 0.001
δ = 0.1 
Nc = floor((0.6745/δ)^2*((norm(f)^2)/(1-norm(T))^2)) + 1

130.0

In [41]:
function mcmcalg1(ϵ, Nc) 
    Xs = fill(0., mT)
    for i in 1:mT
        W_0 = 1
        for s in 1:Nc
            W = W_0; k = 0; point = i; X = W_0 * f[i]
            while abs(W) >= ϵ
                nextpoint  = 1
                u = rand()
                while u >= sum(P[point, 1:nextpoint])  #probabilidad acumulada in-situ
                    nextpoint = nextpoint + 1
                end
                k = k + 1
                    W_new = W *(T[point, nextpoint]/P[point, nextpoint]) # T,P variables globales
                    X = X + W_new * f[nextpoint] # f variables globales
                point = nextpoint
                W = W_new
            end
        Xs[i] += X
        end
    end
    Xs = Xs/Nc
end

mcmcalg1 (generic function with 1 method)

In [42]:
TEalg1 = minimum(@benchmark mcmcalg1(ϵ, Nc))

BenchmarkTools.TrialEstimate: 
  time:             223.753 ms
  gctime:           28.876 ms (12.91%)
  memory:           337.95 MiB
  allocs:           7501525

### MCCM con matriz de probabilidad acumulada fuera del algoritmo, variable global
Se calcula la suma de probabilidad acumulada fuera del algoritmo `mcmc`. Continúa el uso de las matrices `T, Pa, P` y el vector `f` como variables globales.

In [21]:
M = diagm(0 => diag(A))
N = M-A
T = inv(M) * N
f = inv(M) * b
nT, mT = size(T);

In [30]:
S = fill(0, nT)
Pa = P = fill(0., nT, mT)
[S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0]
[P[i,j]= 1/S[i] for i in 1:nT, j in 1:mT if T[i,j] != 0 ]
Pa = [accumulate(+, P[i, 1:mT]) for i in 1:nT]
Pi = [1/nT for i in 1:nT];

In [43]:
function mcmcalg2(ϵ, Nc)
    Xs = fill(0., mT)
    for i in 1:mT
        W_0 = 1.0
        for s in 1:Nc
            W = W_0;  point = i; X = W_0 * f[i]
            while abs(W) >= ϵ
                nextpoint = 1
                u = rand()
                while u >= Pa[point][nextpoint]
                    nextpoint += 1
                end
                    W_new = W *(T[point, nextpoint]/P[point,nextpoint])
                    X += W_new * f[nextpoint]
                point = nextpoint
                W = W_new
            end
        Xs[i] += X
        end
    end
    Xs = Xs/Nc
end

mcmcalg2 (generic function with 1 method)

In [44]:
TEalg2 = minimum(@benchmark mcmcalg2(ϵ, Nc))

BenchmarkTools.TrialEstimate: 
  time:             88.682 ms
  gctime:           1.040 ms (1.17%)
  memory:           64.65 MiB
  allocs:           4237108

### MCCM con variables locales.
A continuación se envuelve en la función el cálculo de las matrices `M,N,T` y el vector `f` por lo que es necesrio enviar como argumentos de la función a la matriz `A` y el vector `b`.

In [45]:
function mcmcalg3(ϵ, δ, A, b) 
    M = diagm(0 => diag(A))
    N = M-A
    T = inv(M) * N
    f = inv(M) * b
    nT, mT = size(T)
    S = fill(0, nT)
    Pa = P = fill(0., nT, mT) 
    [S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0]
    [P[i,j]= 1/S[i] for i in 1:nT, j in 1:mT if T[i,j] != 0 ]
    Pa = [accumulate(+, P[i, 1:mT]) for i in 1:nT]
    Pi = [1/nT for i in 1:nT];
    Nc = floor((0.6745/δ)^2*((norm(f)^2)/(1-norm(T))^2)) + 1
  
    Xs = fill(0., mT)

    for i in 1:mT
        W_0 = 1
        for s in 1:Nc
            W = W_0; k = 0; point = i; X = W_0 * f[i]
            while abs(W) >= ϵ
                nextpoint  = 1
                u = rand()
                while u >= Pa[point][nextpoint]
                    nextpoint = nextpoint + 1
                end
                k = k + 1
                    W_new = W *(T[point, nextpoint]/P[point, nextpoint])
                    X = X + W_new * f[nextpoint]
                point = nextpoint
                W = W_new
            end
        Xs[i] += X
        end
    end
    Xs = Xs/Nc
end

mcmcalg3 (generic function with 1 method)

In [50]:
TEalg3 = minimum(@benchmark mcmcalg3(ϵ, δ, A, b))

BenchmarkTools.TrialEstimate: 
  time:             4.923 ms
  gctime:           0.000 ns (0.00%)
  memory:           66.97 KiB
  allocs:           83

### MCCM con Matriz de probabilidad acumulada pre-build con envío de parámetros y type-anotations
Se usan variables locales, se envÍan como argumentos la matriz `A` y el vector `b`. Se calcula la matriz de probabilidad acumulada fuera del algoritmo `mcmc` y se usan anotaciones de tipos.

In [47]:
function mcmcalg4(ϵ::Float64, δ::Float64, A::Matrix{Float64}, b::Vector{Float64})
    M = diagm(0 => diag(A))
    N = M-A
    T = inv(M) * N
    f = inv(M) * b
    nT, mT = size(T)
    S = fill(0, nT)
    Pa = P = fill(0., nT, mT) 
    [S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0]
    [P[i,j]= 1/S[i] for i in 1:nT, j in 1:mT if T[i,j] != 0 ]
    Pa = [accumulate(+, P[i, 1:mT]) for i in 1:nT]
    Pi = [1/nT for i in 1:nT]
    Nc = floor((0.6745/δ)^2*((norm(f)^2)/(1-norm(T))^2)) + 1
    
    Xs = fill(0., mT)
    for i in 1:mT
        W_0 = 1.0
        for s in 1:Nc
            W = W_0; point = i; X = W_0 * f[i]::Float64
            while abs(W) >= ϵ
                nextpoint  = 1::Int64
                u = rand()::Float64
                while u >= Pa[point][nextpoint]::Float64
                    nextpoint = nextpoint + 1::Int64
                end
                    W_new = W *(T[point, nextpoint]/P[point, nextpoint])::Float64
                    X = X + W_new * f[nextpoint]::Float64
                point = nextpoint::Int64
                W = W_new::Float64
            end
        Xs[i] += X::Float64
        end
    end
    Xs = Xs/Nc::Float64
end

mcmcalg4 (generic function with 1 method)

In [49]:
TEalg4 = minimum(@benchmark mcmcalg4(ϵ, δ, A, b))

BenchmarkTools.TrialEstimate: 
  time:             4.129 ms
  gctime:           0.000 ns (0.00%)
  memory:           66.97 KiB
  allocs:           83